In [18]:
import os
import pickle
import pandas as pd
from sklearn.linear_model import LinearRegression

path = os.getcwd() #get working directory
model = pickle.load(open(path+'/app/models/top_5_players.sav', 'rb'))
player_data = pd.read_csv(path+'/app/data/player_data_single_record.csv')
season_average_data = pd.read_csv(path+'/app/data/season_average_data.csv', index_col=0)

In [30]:
def aggregatePlayerStats(data, player_list):

    team_data = data[data['player'].isin(player_list)] #subset data in the player list
    sum_data = team_data[['fga', '3pa', '2pa', 'fta',
                       'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']].sum()
    avg_data = team_data[['2p%', '3p%', 'fg%', 'ft%', 'efg%', 'age', 'per', '3par',
                         'ftr', 'orb%', 'drb%', 'trb%', 'ast%', 'stl%', 'blk%', 'tov%',
                         'usg%', 'ows', 'dws', 'ws', 'ws/48', 'obpm', 'dbpm', 'bpm', 'vorp']].mean()
    final = pd.DataFrame(pd.concat([sum_data, avg_data], axis=0)).transpose()
    return final

In [37]:
def predictWinPercentage(player_data, season_average_data, player_list, season):
    
    season_data = player_data[player_data['season']==season]
    season_average = season_average_data[season_average_data['season']==season]
    team_stats = aggregatePlayerStats(season_data, player_list)
    team_stats = team_stats - season_average.loc['mean'].drop('season', axis=0) #reflects model type
 
    winning_percentage = float(model.predict(team_stats)) *100
    winning_percentage = round(winning_percentage, 1)
    final = 'Predicted win percentage: ' + str(winning_percentage) + '%'
    return final

In [38]:
player_list = ['Kobe Bryant','Yao Ming','Dwayne Wade','Lebron James','Chris Paul']
predictWinPercentage(player_data, season_average_data, player_list, 2005)

2p%      -0.007139
2pa      -9.821429
3p%       0.002259
3pa      -1.267857
3par      0.012537
age      -2.907407
ast      -1.471429
ast%     13.236111
blk      -1.821429
blk%     -0.876852
bpm       4.469444
dbpm     -0.071296
drb      -7.728571
drb%     -0.599074
dws       1.198148
efg%     -0.022833
fg%      -0.019769
fga     -11.207143
ft%       0.082898
fta      -1.878571
ftr       0.075648
obpm      4.542593
orb      -3.550000
orb%     -2.133333
ows       5.241667
per       7.562963
pf       -4.485714
pts     -12.396429
stl      -0.135714
stl%      1.290741
tov      -2.889286
tov%     -1.391667
trb     -11.257143
trb%     -1.482407
usg%      7.862037
vorp      3.829630
ws        6.436111
ws/48     0.078130
Name: 0, dtype: float64


'Predicted win percentage: 66.4%'